### 深入minst,用cnn来训练

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("e:/tmp/minist_data/", one_hot=True)

Extracting e:/tmp/minist_data/train-images-idx3-ubyte.gz
Extracting e:/tmp/minist_data/train-labels-idx1-ubyte.gz
Extracting e:/tmp/minist_data/t10k-images-idx3-ubyte.gz
Extracting e:/tmp/minist_data/t10k-labels-idx1-ubyte.gz


In [3]:
# tf 计算图输入
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

In [4]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1) #标准差0.1
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

卷积和池化

tf.nn.conv2d(input, filter, strides, padding)

我们的卷积使用1步长（stride size），0边距（padding size）的模板

tf.nn.max_pool(value, ksize, strides, padding, name=None)

value: A 4-D Tensor with shape [batch, height, width, channels] and type float32 ， RGB则是3通道

In [5]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

第一层卷积

现在我们可以开始实现第一层了。它由一个卷积接一个max pooling完成。卷积在每个5x5的patch中算出32个特征(depth=32,32个神经元)。卷积的权重张量形状是[5, 5, 1, 32]，前两个维度是patch的大小，接着是输入的通道数目，最后是输出的通道数目。 而对于每一个输出通道都有一个对应的偏置量。


In [6]:
x_image = tf.reshape(x, [-1,28,28,1])
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

第二层卷积

In [7]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

全连接

图片尺寸减小到7x7， 28/2/2=7

In [8]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

dropout

In [9]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

output

softmax

In [10]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

训练和测试

In [11]:
cross_entropy = -tf.reduce_sum(y*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

cost = tf.reduce_mean(cross_entropy)


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
      batch_x, batch_y = mnist.train.next_batch(50)
      if i%100 == 0:
        train_accuracy = sess.run(accuracy,feed_dict={ x:batch_x, y: batch_y, keep_prob: 1.0})
        #train_accuracy = accuracy.eval(feed_dict={ x:batch_x, y: batch_y, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
      train_step.run(feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})
    
    print("train finished")

    #test
    test_accuracy = accuracy.eval(feed_dict={ x: mnist.test.images[:5000], y: mnist.test.labels[:5000], keep_prob: 1.0})
    print ("test accuracy %g"%test_accuracy) 

step 0, training accuracy 0.1
step 100, training accuracy 0.88
step 200, training accuracy 0.8
step 300, training accuracy 0.94
step 400, training accuracy 0.9
step 500, training accuracy 0.98
step 600, training accuracy 0.88
step 700, training accuracy 0.96


In [15]:
mnist.test.images.shape

(10000, 784)

In [16]:
mnist.test.labels.shape

(10000, 10)